In [31]:
from collections import defaultdict
from enum import Enum
import numpy as np
import re
import os
from typing import Dict, List, NamedTuple, Optional

In [42]:
class Author(Enum):
    PRUS = 0
    SIENKIEWICZ = 1
    ORZESZKOWA = 2

In [19]:
def preprocess(text: str) -> str:
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\ +', ' ', text)
    return text

def load_corpus_list(path: str) -> List[str]:
    text = open(path, 'r').read()
    preprocessed = preprocess(text)
    words = preprocessed.split()
    return words

def occurence_dict_from_corpus(words: List[str]) -> Dict[str, float]:
    unique_words, word_counts = np.unique(words, return_counts=True)
    word_counts_perc = word_counts / np.sum(word_counts)
    return defaultdict(float, zip(unique_words, word_counts_perc))

def create_occurence_dicts() -> Dict[Author, Dict[str, float]]:
    occurence_dicts = dict()
    occurence_dicts[Author.PRUS] = occurence_dict_from_corpus(
        words=load_corpus_list(path='data/korpus_prusa.txt')
    )
    occurence_dicts[Author.SIENKIEWICZ] = occurence_dict_from_corpus(
        words=load_corpus_list(path='data/korpus_sienkiewicza.txt')
    )
    occurence_dicts[Author.ORZESZKOWA] = occurence_dict_from_corpus(
        words=load_corpus_list(path='data/korpus_orzeszkowej.txt')
    )
    return occurence_dicts

### Zadanie 1. (3+1+Xp) 
Napisz program, który ustala autora zdania. Autorów jest trójka: Prus, Orzeszkowa, Sienkiewicz (POS), na SKOSie znajdziesz odpowiednie dane uczące. Powinieneś je podzielić na część uczącą i walidacyjną. Część X punktacji zależeć będzie od tego, jak Twój program wypadnie na tle innych programów dla danych testowych (które pojawią się później na SKOSie). Dwie podstawowe metody są następujące:

a) Naive Bayes (będzie na wykładzie 4, powinieneś użyć przynajmniej jednej cechy, która nie jest słowem)

b) Utworzenie trzech modeli językowych i wybór tego, który daje najepszy wynik na klasyfikowanym zdaniu.

Za sprawdzenie dwóch podejść jest punkt premiowy.

In [37]:
def find_author(text: str, occurence_dicts: Dict[Author, Dict[str, float]]) -> Author:
    scores: List[float] = []
    for author, occurence_dict in occurence_dicts.items():
        words = preprocess(text=text).split()
        score = np.sum(np.log([occurence_dict[word] + 1e-5 for word in words]))
        scores.append(score)
        
    author = list(occurence_dicts.keys())[np.argmax(scores)]
    return author

In [43]:
def test_classifier(folder_path: str) -> np.ndarray:
    '''
    Returns confussion matrix
    '''
    occurence_dicts = create_occurence_dicts()
    
    authors_number = len(list(occurence_dicts.keys()))
    confussion_matrix = np.zeros((authors_number, authors_number))
    
    for relative_path in os.listdir(folder_path):
        path = os.path.join(folder_path, relative_path)
        
        if not os.path.isfile(path):
            continue
        
        true_author: Optional[Author] = None
        
        if 'orzeszkowej' in relative_path:
            true_author = Author.ORZESZKOWA
        if 'prusa' in relative_path:
            true_author = Author.PRUS
        if 'sienkiewicza' in relative_path:
            true_author = Author.SIENKIEWICZ
        
        if true_author is None:
            print(f'Could not find a true author of the text! Filename: {relative_path}')
            continue
        
        text = open(path, 'r').read()
        classified_author = find_author(text=text, occurence_dicts=occurence_dicts)
        
        confussion_matrix[true_author.value, classified_author.value] += 1
    
    return confussion_matrix

In [54]:
confussion_matrix = test_classifier(folder_path='testy1')
authors = list(map(lambda author: author.name, list(Author)))
authors, confussion_matrix

(['PRUS', 'SIENKIEWICZ', 'ORZESZKOWA'],
 array([[21.,  0.,  0.],
        [15., 12.,  0.],
        [ 0.,  0., 12.]]))

### Zadanie 2. (3+Xp) 
W zadaniu tym powinieneś uporządkować (spermutować) ciąg słów, żeby utworzył zdanie. W stosunku do poprzedniej listy mamy następujące różnice:

a) powinieneś wykorzystać tagi słów (na przykład z pliku supertags.txt, link na SKOSie)

b) powinieneś je połączyć ze zwykłymi statystykami bigramowymi (lub, opcjonalnie, z sufiksami)

c) powinieneś wyodrębnić z danych uczących część walidacyjną i dobrać parametry łączenia modeli bazujących na słowach i bazujących na tagach.

Sposób oceny będzie taki sam, jak w przypadku zadania z P1. Wybór zdań do oceny pojawi się przed zajęciami (będą to zdania zawierające od 4 do 8 tokenów, wybrane losowo z części testowej korpusu PolEval)

### Zadanie 3. (4p) 
W zadaniu tym powinieneś losować zdania o słowach z identyczną charakterystyką gramatyczną jak zdanie wejściowe). Przykładowo dla zdania:

Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę. wynikiem mogłoby być
Gruby Stefan przeczytał we wczorajszej gazecie starannie przygotowaną analizę.
Zgodność gramatyczną sprawdzamy za pomocą tagów z pliku supertags. Przyjmijmy, że słowo s niewystępujące w tym pliku ma opis gramatyczny (’^’ + s)[-3:]. Powinieneś korzystać ze statystyk unigramowych.

### Zadanie 4. (3p) 
Dodaj statystyki bigramowe do powyższego zadania. Postaraj się, by jak najrza- dziej zdażały się sytuacje, w których musisz losować posługując się unigramami. Zaznaczaj znakiem "|" każdą taką nieciągłość.

### Zadanie 5. (5+1p) 
W zadaniu tym zajmiemy się kolokacjami słów, o których mamy informacje gramatyczną. Napisz program, który dla danego słowa znajduje k najbardziej z nim „spokrewnio- nych” słów. Rozważ następujące metody wyznaczania kolokacji:

a) PPMI (Positive Pointwise Mutual Information)

b) jakiś inny, dowolnie wybrany, z używanych na kolokacje wzorów (więcej na wykładzie),

c) kolokacje gramatyczno-słowowe (tzn. żeby dwa słowa były uznane za kolokacje, warunek kolo- kacyjności (dowolnie wybrany) powinny spełniać zarówno tagi słów, jak i same słowa.

d) jakaś dowolna inna metoda, lub Twoja modyfikacja powyższych (za to dodatkowy punkt).

Możesz się ograniczyć do słów, które są stosunkowo częste (więcej niż n wystąpień w korpusie) i występują co najmniej raz w jakimś trigramie (lub k razy w jakimś bigramie). Wybierz niewielki zbiór słów (powiedzmy koło 10). Przygotuj raport, w którym dla każdego z tych słów jest 10 najbardziej spokrewnionych słow (czyli takich, o największym współczynniku kolokacji), dla różnych metod wyznaczania kolokacji.

### Zadanie 6. (7p) 
W zadaniu tym będziemy tworzyć pierwszą wersję programu tworzącego poezję (przypominającą Pana Tadeusza, oznaczanego dalej PT)). Przypomnijmy najważniejsze fakty od- noszące się do tego utworu (i ogólnie Poezji):

F1. Wiersz składa się z wersetów, z których każdy ma ustaloną liczbę sylab (w PT to 13)

F2. Ostatnie słowo w wersecie n rymuje się z ostatnim słowem w wersecie n + 1 (dla n parzystego,
numeracja od 0).

F3. Rym to zgodność ostatniej sylaby i części od samogłoski sylaby przedostatniej (zdrowie-dowie). W zasadzie rymy to zjawisko fonetyczne, ale dla języka polskiego (w pierwszej wersji) można sobie to trochę uprościć i powiedzieć, że dotyczą one liter.

F4. Akcenty słów i podziały słów muszą się jakoś sensownie układać. Nam wystarczy przyjąć, że godzimy się jedynie na takie podziały wersu na słowa k-sylabowe3, których użył Adam Mickiewicz, na przykład:

Litwo, Ojczyzno moja, Ty jesteś jak zdrowie, ile cię trzeba cenić, ten tylko się dowie ma schemat: [2,3,2,1,2,1,2] -- [2,1,2,2,1,2,1,2]

F5. Podział na sylaby nie jest trywialny (dlaczego?). Ale na nasze szczęście policzenie sylab jest łatwe. Słowo ma tyle sylab, ile ma samogłosek (przy czym połączenia ie, iu, ię, itd traktujemy jako jedną samogłoskę). Część rymowana wyrazu to część wyrazu od przedostatniej samogłoski do końca.

Powinieneś stworzyć program, generujący dwuwersowe fragmenty wierszy w stylu PT, czyli powinieneś przypilnować:

a) żeby wersy były poprawne rytmicznie i się rymowały,

b) żeby dwuwers miał sens gramatyczny (czyli by tagi słów pasowały do jakiegoś zdania lub frag- mentu zdania)

c) żeby były jakoś wykorzystane statystyki N-gramowe (plan minimum to statystyki 1-gramowe, dodatkowe +1 za wykorzystanie bigramów).

### Zadanie 7. (5p) 
Zmodyfikuj algorytm z poprzedniego zadania w ten sposób, by starał się on maksymalizować wzajemną „kolokacyjność” słów z dwuwersu (tak, by jak najwięcej słów było ze sobą powiązanych, na przykład przez wysokie PPMI). Akceptowalna jest dowolna procedura (local search, jakieś błądzenie losowe, metody ewolucyjne, ...), która daje wartość liczby par słów będących kolokacjami istotnie większą niż losowanie z poprzedniego zadania.
